# Business Growth Performance

## Import Data Source

In [13]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import reduce
import datetime

mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

In [14]:
as_week = 2023024
data_src = f"../../data/raw/Raw-Agg Perf_Wk {as_week}.csv"
df_bg = pd.read_csv(data_src)

cols = ['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'AGG_TYPE', 'ACTUAL_SNAP', 'ACTUAL_AGG', 'TARGET_SNAP', 'TARGET_AGG', 'PPN_TM']
df_bg = df_bg[cols]
df_bg.head(3)

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,METRIC_CD,METRIC_NAME,AGG_TYPE,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,PPN_TM
0,20230301,2023009,202303,20231,2023,B2R000100,Postpaid Revenue,S,1.376380e+07,1.376380e+07,6.416632e+06,6.416632e+06,2023-06-21 17:48:01
1,20230302,2023009,202303,20231,2023,B2R000100,Postpaid Revenue,S,8.521882e+08,8.659520e+08,8.645637e+08,8.709803e+08,2023-06-21 17:48:01
2,20230303,2023009,202303,20231,2023,B2R000100,Postpaid Revenue,S,9.943611e+07,9.653881e+08,9.872504e+07,9.697053e+08,2023-06-21 17:48:01


In [15]:
df_bg.shape

(550, 13)

In [16]:
df_bg['METRIC_NAME'].unique()

array(['Postpaid Revenue', 'Prepaid Revenue',
       'TDG Revenue (Performance View)', 'TOL Revenue', 'TVS Revenue'],
      dtype=object)

## Data Preporation

In [17]:
''' Function '''

def unit_mb(baht):
    n = baht
    unit_mb = n / 1000000
    return unit_mb

In [18]:
''' Rawdata '''

''' Modify '''
df_bg = df_bg.rename(columns={'ACTUAL_SNAP':'ACT_SNAP', 'ACTUAL_AGG':'ACT_AGG', 'TARGET_SNAP':'TGT_SNAP', 'TARGET_AGG':'TGT_AGG'})

''' Add new column '''
df_bg['MTH_DESC'] = pd.to_datetime(df_bg['TM_KEY_MTH'], format='%Y%m', errors='coerce').dt.strftime('%B')
df_bg['WK_LABEL'] = df_bg['TM_KEY_WK'].apply(str).str[5:].astype(int)
df_bg['DAY_LABEL'] = df_bg['TM_KEY_DAY'].apply(str).str[6:].astype(int)

In [19]:
''' Create Dataframe '''

''' Aggregate type : S '''
df_agg_s = df_bg[df_bg['AGG_TYPE']=='S']

''' Aggregate type : N '''
df_agg_n = df_bg[df_bg['AGG_TYPE']=='N'].dropna(subset='ACT_SNAP')

In [20]:
''' Period Values '''

curr_yr = df_bg['TM_KEY_YR'].max()
last_yr = df_bg['TM_KEY_YR'].max() - 1
curr_mth = df_bg['TM_KEY_MTH'].max()
curr_mth_desc = pd.to_datetime(curr_mth, format='%Y%m', errors='coerce').strftime('%B')
last_mth = int(f'{str(last_yr)}12') if str(curr_mth)[4:]=='01' else curr_mth-1
curr_wk = df_bg['TM_KEY_WK'].max()
last_wk = df_bg['TM_KEY_WK'].drop_duplicates().shift().max()
curr_wk_label = df_bg['WK_LABEL'].iloc[-1]
curr_day = df_bg['TM_KEY_DAY'].max()
last_day = df_bg['TM_KEY_DAY'].drop_duplicates().shift().max()
curr_day_label = df_bg['DAY_LABEL'].iloc[-1]
last_ppn_tm = df_bg['PPN_TM'].max()

## ** Yearly **

### YR : Aggregation

In [31]:
''' Create Dataframe '''

df_agg_s_yr = df_agg_s.groupby(['METRIC_CD', 'METRIC_NAME', 'TM_KEY_YR', 'AGG_TYPE']).agg({'ACT_SNAP': 'sum', 'TGT_SNAP': 'sum'}).reset_index()
df_agg_n_yr = df_agg_n.loc[df_agg_n.groupby('TM_KEY_YR')['TM_KEY_DAY'].idxmax(), ['METRIC_CD', 'METRIC_NAME', 'TM_KEY_YR', 'AGG_TYPE', 'ACT_SNAP', 'TGT_SNAP']]
df_rev_yr = pd.concat([df_agg_s_yr, df_agg_n_yr])

''' YTD '''
df_rev_yr = df_rev_yr[df_rev_yr['TM_KEY_YR']==curr_yr]

''' Aggregation '''
df_rev_yr['BASELINE'] = df_rev_yr.groupby('METRIC_CD')['ACT_SNAP'].shift().fillna(0)
df_rev_yr['%ACHV'] = (df_rev_yr['ACT_SNAP']-df_rev_yr['BASELINE']) / (df_rev_yr['TGT_SNAP']-df_rev_yr['BASELINE'])
df_rev_yr['%ACHV'] = np.where(df_rev_yr['%ACHV'] < 1.2, df_rev_yr['%ACHV'], 1.2)

df_rev_yr

,METRIC_CD,METRIC_NAME,TM_KEY_YR,AGG_TYPE,ACT_SNAP,TGT_SNAP,BASELINE,%ACHV
0,B1R000100,Prepaid Revenue,2023,S,1.317501e+10,1.322304e+10,0.0,0.996368
1,B2R000100,Postpaid Revenue,2023,S,2.361662e+10,2.388398e+10,0.0,0.988806
2,TB3R000100,TOL Revenue,2023,S,4.906520e+09,5.399463e+09,0.0,0.908705
3,TB4R000100,TVS Revenue,2023,S,1.065706e+09,1.404106e+09,0.0,0.758993
323,TNSC00142,TDG Revenue (Performance View),2023,N,5.320000e+08,4.600000e+08,0.0,1.156522


## ** Monthly **

### MTH : Aggregation

In [30]:
''' Create Dataframe '''

df_agg_s_mth = df_agg_s.groupby(['METRIC_CD', 'METRIC_NAME', 'TM_KEY_MTH', 'MTH_DESC', 'AGG_TYPE']).agg({'ACT_SNAP': 'sum', 'TGT_SNAP': 'sum'}).reset_index()
df_agg_n_mth = df_agg_n.loc[df_agg_n.groupby('TM_KEY_MTH')['TM_KEY_DAY'].idxmax(), ['METRIC_CD', 'METRIC_NAME', 'TM_KEY_MTH', 'MTH_DESC', 'AGG_TYPE', 'ACT_SNAP', 'TGT_SNAP']]
df_rev_mth = pd.concat([df_agg_s_mth, df_agg_n_mth])

''' MTD '''
df_rev_mth = df_rev_mth[df_rev_mth['TM_KEY_MTH']==curr_mth]

''' Aggregation '''
df_rev_mth['BASELINE'] = df_rev_mth.groupby('METRIC_CD')['ACT_SNAP'].shift().fillna(0)
df_rev_mth['%ACHV'] = (df_rev_mth['ACT_SNAP']-df_rev_mth['BASELINE']) / (df_rev_mth['TGT_SNAP']-df_rev_mth['BASELINE'])
df_rev_mth['%ACHV'] = np.where(df_rev_mth['%ACHV'] < 1.2, df_rev_mth['%ACHV'], 1.2)

df_rev_mth

,METRIC_CD,METRIC_NAME,TM_KEY_MTH,MTH_DESC,AGG_TYPE,ACT_SNAP,TGT_SNAP,BASELINE,%ACHV
3,B1R000100,Prepaid Revenue,202306,June,S,2.173048e+09,2.200244e+09,0.0,0.987640
7,B2R000100,Postpaid Revenue,202306,June,S,3.776692e+09,3.920805e+09,0.0,0.963244
11,TB3R000100,TOL Revenue,202306,June,S,8.272998e+08,8.789668e+08,0.0,0.941218
15,TB4R000100,TVS Revenue,202306,June,S,1.436620e+08,1.632565e+08,0.0,0.879977
323,TNSC00142,TDG Revenue (Performance View),202306,June,N,5.320000e+08,4.600000e+08,0.0,1.156522


## ** Weekly **

### WK : Aggregation

In [32]:
''' Create Dataframe '''

df_agg_s_wk = df_agg_s.groupby(['METRIC_CD', 'METRIC_NAME', 'TM_KEY_WK', 'WK_LABEL', 'AGG_TYPE']).agg({'ACT_SNAP': 'sum', 'TGT_SNAP': 'sum'}).reset_index()
df_agg_n_wk = df_agg_n.loc[df_agg_n.groupby('TM_KEY_WK')['TM_KEY_DAY'].idxmax(), ['METRIC_CD', 'METRIC_NAME', 'TM_KEY_WK', 'WK_LABEL', 'AGG_TYPE', 'ACT_SNAP', 'TGT_SNAP']]
df_rev_wk = pd.concat([df_agg_s_wk, df_agg_n_wk])

''' WTD '''
df_rev_wk = df_rev_wk[df_rev_wk['TM_KEY_WK']==last_wk]

''' Aggregation '''
df_rev_wk['BASELINE'] = df_rev_wk.groupby('METRIC_CD')['ACT_SNAP'].shift().fillna(0)
df_rev_wk['%ACHV'] = (df_rev_wk['ACT_SNAP']-df_rev_wk['BASELINE']) / (df_rev_wk['TGT_SNAP']-df_rev_wk['BASELINE'])
df_rev_wk['%ACHV'] = np.where(df_rev_wk['%ACHV'] < 1.2, df_rev_wk['%ACHV'], 1.2)

df_rev_wk

,METRIC_CD,METRIC_NAME,TM_KEY_WK,WK_LABEL,AGG_TYPE,ACT_SNAP,TGT_SNAP,BASELINE,%ACHV
14,B1R000100,Prepaid Revenue,2023023,23,S,8.347299e+08,8.565297e+08,0.0,0.974549
30,B2R000100,Postpaid Revenue,2023023,23,S,1.363180e+09,1.406488e+09,0.0,0.969208
46,TB3R000100,TOL Revenue,2023023,23,S,3.198776e+08,3.325761e+08,0.0,0.961818
62,TB4R000100,TVS Revenue,2023023,23,S,4.736705e+07,5.325349e+07,0.0,0.889464
316,TNSC00142,TDG Revenue (Performance View),2023023,23,N,5.400000e+08,4.600000e+08,0.0,1.173913
